In [3]:
import cPickle as pickle
import pickle
from datetime import datetime
import os
import sys
from lasagne import layers
from nolearn.lasagne import BatchIterator
from nolearn.lasagne import NeuralNet
from pandas import DataFrame
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
import theano
from scipy import misc
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import glob
from PIL import Image
import os

In [5]:
ellipse_fs = glob.glob('/home/abdullah/PycharmProjects/Pattern/Ellipse_TS/*.bmp')
line_fs = glob.glob('/home/abdullah/PycharmProjects/Pattern/Ellipse_TS/*.bmp')
Diamond_fs = glob.glob('/home/abdullah/PycharmProjects/Pattern/Ellipse_TS/*.bmp')

x = np.array([np.array(Image.open(fname)) for fname in ellipse_fs])
x = np.concatenate((x,np.array([np.array(Image.open(fname)) 
                                for fname in line_fs])),axis=0)
x = np.concatenate((x,[np.array(Image.open(fname)) 
                    for fname in Diamond_fs]),axis=0).astype('float32')

y=np.zeros((255,))
y[0:85]=1.0
y[85:170]=2.0
y[170:]=3.0
# x_data = open('x_data.p', 'wb')
# y_data = open('y_data.p', 'wb')
# pickle.dump( x, x_data, protocol=2 )
# pickle.dump( y, y_data, protocol=2 )
# x = pickle.load(open("x_data.p",'r'))
# y = pickle.load(open("y_data.p",'r')) 
X_train, X_test, y_train, y_test = train_test_split\
    (x, y, test_size=0.2, random_state=42)
X_train, X_eval, y_train, y_eval = train_test_split\
    (X_train, y_train, test_size=0.25, random_state=42)

In [2]:
FTRAIN = '/facial_points_training.csv'
FTEST = '/facial_points_test.csv'
FLOOKUP = '/IdLookupTable.csv'
def make_submission(test_labels):
    test_labels *= 48.0
    test_labels += 48.0
    test_labels = test_labels.clip(0, 96)
    lookup_table = pd.read_csv(FLOOKUP)
    values = []

    cols = joblib.load('cols.pkl')

    for index, row in lookup_table.iterrows():
        values.append((
            row['RowId'],
            test_labels[row.ImageId - 1][np.where(cols == row.FeatureName)[0][0]],
            ))
    submission = pd.DataFrame(values, columns=('RowId', 'Location'))
    submission.to_csv('CSVs/submission.csv', index=False)
    
    
    
def load(test=False, cols=None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe
    
    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    cols_l = df.columns[:-1]

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]
    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
        joblib.dump(cols_l, 'cols.pkl', compress=3)
    else:
        y = None

    return X, y

In [4]:
def load2d(test=False, cols=None):
    X, y = load(test=test,cols=cols)
    X = X.reshape(-1, 1, 96, 96)
    return X, y
colos=['left_eye_center_x',
      'left_eye_center_y',
      'right_eye_center_x',
      'right_eye_center_y',
        'nose_tip_x',
      'nose_tip_y',
      'mouth_center_bottom_lip_x',
      'mouth_center_bottom_lip_y']

In [6]:
class FlipBatchIterator(BatchIterator):
    flip_indices = [
        (0, 2), (1, 3),
        (4, 8), (5, 9), (6, 10), (7, 11),
        (12, 16), (13, 17), (14, 18), (15, 19),
        (22, 24), (23, 25),
        ]

def transform(self, Xb, yb):
    Xb, yb = super(FlipBatchIterator, self).transform(Xb, yb)

    # Flip half of the images in this batch at random:
    bs = Xb.shape[0]
    indices = np.random.choice(bs, bs / 2, replace=False)
    Xb[indices] = Xb[indices, :, :, ::-1]

    if yb is not None:
        # Horizontal flip of all x coordinates:
        yb[indices, ::2] = yb[indices, ::2] * -1

        # Swap places, e.g. left_eye_center_x -> right_eye_center_x
        for a, b in self.flip_indices:
            yb[indices, a], yb[indices, b] = (
                yb[indices, b], yb[indices, a])

    return Xb, yb
class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = np.float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)

In [7]:
from lasagne import layers
from lasagne.updates import adam
from nolearn.lasagne import NeuralNet
from lasagne.layers import Conv2DLayer,batch_norm,BatchNormLayer
from nolearn.lasagne import BatchIterator
from lasagne.init import HeNormal
net = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        #('b1',layers.BatchNormLayer),
        ('dropout1', layers.DropoutLayer),  # !
        ('conv2', layers.Conv2DLayer),
        #('b2',layers.BatchNormLayer),
        ('dropout2', layers.DropoutLayer),  # !
        ('conv3', layers.Conv2DLayer),
        #('b3',layers.BatchNormLayer),
        ('dropout3', layers.DropoutLayer),  # !
        ('conv6', layers.Conv2DLayer),
        ('dropout6', layers.DropoutLayer),  # !
        ('hidden4', layers.DenseLayer),
        #('b4',layers.BatchNormLayer),
        ('dropout4', layers.DropoutLayer),  # !
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, 96, 96),
    conv1_W =HeNormal(gain='relu'), conv1_num_filters=16, conv1_filter_size=(5, 5),conv1_pad='same', dropout1_p=0.2,  # !
    conv2_W =HeNormal(gain='relu'), conv2_num_filters=32, conv2_filter_size=(3, 3),conv2_pad='same', dropout2_p=0.2,  # !
    conv3_W =HeNormal(gain='relu'), conv3_num_filters=32, conv3_filter_size=(3, 3),conv3_pad='same', dropout3_p=0.2,  # !
    conv6_W =HeNormal(gain='relu'), conv6_num_filters=16, conv6_filter_size=(1, 1),conv6_pad='same', dropout6_p=0.2,  # !
    hidden4_num_units=64, dropout4_p=0.2,  # !
    hidden5_num_units=128,
    output_num_units=30, output_nonlinearity=None,

    update_learning_rate=theano.shared(np.float32(0.03)),

    regression=True,
    batch_iterator_train=FlipBatchIterator(batch_size=128),
    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.03, stop=0.0001)
    ],
    max_epochs=3000,
    verbose=1,
    )

net.fit(X, y)

# Neural Network with 9464270 learnable parameters

## Layer information

  #  name      size
---  --------  --------
  0  input     1x96x96
  1  conv1     16x96x96
  2  dropout1  16x96x96
  3  conv2     32x96x96
  4  dropout2  32x96x96
  5  conv3     32x96x96
  6  dropout3  32x96x96
  7  conv6     16x96x96
  8  dropout6  16x96x96
  9  hidden4   64
 10  dropout4  64
 11  hidden5   128
 12  output    30

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  ------
      1       0.18987       0.02985      6.36014  17.90s
      2       0.07353       0.03141      2.34080  17.96s
      3       0.02026       0.01558      1.30023  17.87s
      4       0.00879       0.00986      0.89116  17.92s
      5       0.00708       0.01212      0.58396  17.91s
      6       0.00651       0.01110      0.58640  17.87s
      7       0.00609       0.00991      0.61513  17.89s
      8       0.00585       0.00925      0.63268  17.89s
      9       0.00552     

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7fb66c078f50>,
     batch_iterator_train=<__main__.FlipBatchIterator object at 0x7fb6579b5550>,
     conv1_W=<lasagne.init.HeNormal object at 0x7fb6579b54d0>,
     conv1_filter_size=(5, 5), conv1_num_filters=16, conv1_pad='same',
     conv2_W=<lasagne.init.HeNormal object at 0x7fb6579b5510>,
     conv2_filter_size=(3, 3), conv2_num_filters=32, conv2_pad='same',
     conv3_W=<lasagne.init.HeNormal object at 0x7fb6579b5450>,
     conv3_filter_size=(3, 3), conv3_num_filters=32, conv3_pad='same',
     conv6_W=<lasagne.init.HeNormal object at 0x7fb6579b5490>,
     conv6_filter_size=(1, 1), conv6_num_filters=16, conv6_pad='same',
     custom_scores=None, dropout1_p=0.2, dropout2_p=0.2, dropout3_p=0.2,
     dropout4_p=0.2, dropout6_p=0.2, hidden4_num_units=64,
     hidden5_num_units=128, input_shape=(None, 1, 96, 96),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('conv1

In [8]:
import cPickle as pickle
with open('net.pickle', 'wb') as f:
    pickle.dump(net, f, -1)

In [16]:
import csv
import pickle
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from sklearn.externals import joblib
from skimage.transform import rotate
from skimage.util import random_noise
import pandas as pd
from sklearn.externals import joblib
X1, y1 = load2d()
tst = load(test=True)[0].reshape(-1, 1,96, 96)
y_pred1 = net.predict(tst)
print y_pred1.shape, tst.shape
make_submission(y_pred1)

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b